In [2]:
import os
import shutil
try:
    shutil.rmtree("generated_images")
    os.mkdir("generated_images")
except:
    os.mkdir("generated_images")

In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np
from MIDIConverter import MIDIConverter
from music21 import instrument, note, chord, stream, converter
import sys
from imageio import imwrite

In [5]:
# Generating Dataset


# os.makedirs("generated_images")
cvt = MIDIConverter()
parent_path = "/kaggle/input/classical-music-midi/mozart"

all_files = []

for root, dirnames, filenames in os.walk(parent_path):
    for j in filenames:
        all_files.append(root + "/" + j)


for fname in all_files:
    try:
        cvt.midi2image(fname)
    except:
        pass


/opt/conda/lib/python3.7/site-packages/music21/midi/translate.py:885: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2005 by Bernd Kr\xfcger'>; getting generic Instrument
  TranslateWarning)
/opt/conda/lib/python3.7/site-packages/music21/midi/translate.py:885: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Copyright \xa9 2004 by Bernd Kr\xfcger'>; getting generic Instrument
  TranslateWarning)
/opt/conda/lib/python3.7/site-packages/music21/midi/translate.py:885: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Krueger'>; getting generic Instrument
  TranslateWarning)
/opt/conda/lib/python3.7/site-packages/music21/midi/translate.py:885: TranslateWarning: Unable to determine instrument from <music21.midi.

UnidentifiedImageError: cannot identify image file '/kaggle/input/classical-music-midi/liszt/liz_et6.mid'

In [6]:
path = '/kaggle/working/generated_images'
os.getcwd()
img_list = os.listdir(path)


def image_data(img_list, path, length):
    pixels = []
    imgs = []    
    for img in img_list:
        if "png" in img:
            # quantized and dithered image
            img_arr = Image.open(path+'/'+img, 'r').convert('1') 
            # normalize the pixel values
            pixel = np.array(img_arr.getdata()).astype('float32') / 255.0 
            # pad with zeros
            pixel = np.pad(pixel, (0, 106 * 106 - pixel.shape[0]), 'constant', constant_values=(0))
            # reshaping into 106 to 106
            pixels.append(pixel.reshape((106, 106, 1)))
            imgs.append(img_arr)
    return np.array(pixels), imgs




def show_image(pixels):
    array = np.array(pixels.reshape(106, 106), dtype=np.uint8)
    new_image = Image.fromarray(array)
    new_image.show()


# pixels, imgs = access_images(img_list, path, 200)
pixels, imgs = image_data(img_list, path, 200)

In [7]:
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten, BatchNormalization
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from IPython.display import clear_output

In [8]:

def define_discriminator(in_shape=(106, 106, 1)):
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
    
    
    return model
def define_generator(latent_dim):
    model = tf.keras.Sequential()
    n_nodes = 128 * 53 * 53
    model.add(tf.keras.layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(Reshape((53, 53, 128)))
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Conv2DTranspose(1024, (4, 4), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid'))
    return model

def define_gan(g_model, d_model):
    d_model.trainable = False
    model = tf.keras.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [9]:

# Generate Real and Fake Samples to test the discriminator

def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    # Return X and y
    return dataset[ix], np.ones((n_samples, 1))

def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y


def generate_latent_points(latent_dim, n_samples):
    # generate values from normal distribution of shape 106 x 106
    x_input = randn(latent_dim * n_samples).reshape(n_samples, latent_dim)
    return x_input



In [10]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=35, n_batch=10):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    n_samples = 100
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # get discriminator loss
            d_loss, _ = d_model.train_on_batch(X, y)
            # get generator loss
            g_loss = gan_model.train_on_batch(generate_latent_points(latent_dim, n_batch), ones((n_batch, 1)))
            print('>%d, %d/%d, d=%.3f, g=%.3f' %
                  (i+1, j+1, bat_per_epo, d_loss, g_loss))
        if (i+1) % 10 == 0:
            # evaluate performance of discriminator and generator
            X_real, y_real = generate_real_samples(dataset, n_samples)

            _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)

            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
            _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose=0)
            print('>Accuracy real: %.0f%%, fake: %.0f%%' %
                  (acc_real*100, acc_fake*100))
            filename = 'generator_model_%03d.h5' % (epoch + 1)
            g_model.save(filename)
            clear_output()

In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
train(g_model, d_model, gan_model, np.array(pixels), latent_dim)

1/1 [==============================] - 10s 10s/step
>1, 1/43, d=0.795, g=1.662
1/1 [==============================] - 9s 9s/step


In [ ]:
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot


# Generating the Output

def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


model = g_model
# latent_points = generate_latent_points(latent_dim, 1)
# X = g_model.predict(latent_points)
# array = np.array(X.reshape(pixel_width, pixel_width), dtype=np.uint8)
# array *= 255
# new_image = Image.fromarray(array, 'L')
# new_image = new_image.save('/kaggle/working/test/composition.png')

output_notes = []

for i in range(10): 
    latent_points = generate_latent_points(latent_dim, 1)
    X = g_model.predict(latent_points)
    array = np.array(X.reshape(pixel_width, pixel_width), dtype=np.uint8)
    array *= 255
    new_image = Image.fromarray(array, 'L')
    new_image = new_image.save('/kaggle/working/test/composition_{}.png'.format(i))
    output_notes+=cvt.image2midi('/kaggle/working/test/composition_{}.png'.format(i))
    
image_path = '/kaggle/working/composition.mid'
print("Output Notes: ", output_notes)
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp=image_path)